In [1]:
import numpy as np
import tensorflow as tf
import spacetrack.operators as op
from datetime import datetime,timedelta
import pandas as pd
from spacetrack import SpaceTrackClient
from io import StringIO  # Import StringIO
import time
import matplotlib.pyplot as plt
import tqdm
import glob

2025-11-26 22:20:21.031186: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-26 22:20:22.473991: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-26 22:20:22.767766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764170423.167015   47231 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764170423.251793   47231 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-26 22:20:24.503247: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
resample_tle_df = pd.read_csv('../sample_dataset/sample_daily_records_data.csv')
resample_tle_df.set_index('Epoch', inplace=True)
resample_tle_df.head()

,First Derivative Mean Motion,Inclination (degrees),Right Ascension of the Ascending Node (degrees),Argument of Perigee (degrees),Mean Anomaly (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch
Epoch,,,,,,,,
2022-12-31,0.000157,51.644800,77.042350,208.400550,320.463100,0.000525,15.497978,37584.500000
2023-01-01,0.000166,51.644567,74.151133,211.103567,277.293067,0.000519,15.498221,37593.666667
2023-01-02,0.000175,51.644680,69.554840,215.693640,221.644740,0.000511,15.498584,37608.000000
2023-01-03,0.000162,51.644860,64.713420,219.864140,202.583880,0.000499,15.498874,37623.200000
2023-01-04,0.000142,51.644740,59.072260,225.670000,219.529780,0.000497,15.499123,37640.800000


# Feature preparation

In [3]:
# --- 1. Prepare Sample Data ---
data_for_training = resample_tle_df.copy()

## First differencing

In [4]:
data_for_training['Revolution Number at Epoch_Diff'] =  data_for_training['Revolution Number at Epoch'].diff()
initial_revs = data_for_training['Revolution Number at Epoch'][0]

print(f'Initial revs : {initial_revs}' )
data_for_training.head()

Initial revs : 37584.5


/tmp/ipykernel_47231/2648655564.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_revs = data_for_training['Revolution Number at Epoch'][0]


,First Derivative Mean Motion,Inclination (degrees),Right Ascension of the Ascending Node (degrees),Argument of Perigee (degrees),Mean Anomaly (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch,Revolution Number at Epoch_Diff
Epoch,,,,,,,,,
2022-12-31,0.000157,51.644800,77.042350,208.400550,320.463100,0.000525,15.497978,37584.500000,NaN
2023-01-01,0.000166,51.644567,74.151133,211.103567,277.293067,0.000519,15.498221,37593.666667,9.166667
2023-01-02,0.000175,51.644680,69.554840,215.693640,221.644740,0.000511,15.498584,37608.000000,14.333333
2023-01-03,0.000162,51.644860,64.713420,219.864140,202.583880,0.000499,15.498874,37623.200000,15.200000
2023-01-04,0.000142,51.644740,59.072260,225.670000,219.529780,0.000497,15.499123,37640.800000,17.600000


# Cyclical features

In [5]:
def encode_cyclical_feature(df: pd.DataFrame, column_name: str, period: float = 360.0) -> pd.DataFrame:
    """
    Encodes a cyclical feature (like degrees, hours, or months) using
    sine and cosine projection.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the cyclical feature column (in degrees).
        period (float): The length of the cycle (e.g., 360 for degrees, 24 for hours).
                        Defaults to 360.0.

    Returns:
        pd.DataFrame: A new DataFrame with two added columns:
                      <column_name>_sin and <column_name>_cos.
    """
    print(f"--- Encoding '{column_name}' with period {period} ---")

    # 1. Conversion to Radians: Necessary for NumPy's sin/cos functions.
    # The formula converts the value into a proportion of the cycle (0 to 2*pi).
    angle_rad = 2 * np.pi * df[column_name] / period

    # 2. Sine and Cosine Projection
    sin_col = f'{column_name}_sin'
    cos_col = f'{column_name}_cos'

    df[sin_col] = np.sin(angle_rad)
    df[cos_col] = np.cos(angle_rad)

    # Optional: Drop the original column if you don't need it.
    # df = df.drop(columns=[column_name])

    print(f"Created columns: '{sin_col}' and '{cos_col}'")
    return df

def decode_cyclical_feature(df: pd.DataFrame, sin_col: str, cos_col: str, period: float = 360.0) -> pd.DataFrame:
    """
    Reconstructs the original cyclical feature value (in degrees) from its
    sine and cosine components.

    Args:
        df (pd.DataFrame): The input DataFrame containing sine and cosine components.
        sin_col (str): The name of the sine component column.
        cos_col (str): The name of the cosine component column.
        period (float): The length of the cycle (e.g., 360 for degrees, 24 for hours).
                        Defaults to 360.0.

    Returns:
        pd.DataFrame: The DataFrame with a new reconstructed column.
    """
    print(f"\n--- Decoding from '{sin_col}' and '{cos_col}' ---")
    reconstructed_col = f'{sin_col.split("_sin")[0]}_reconstructed'

    # 1. Use arctan2 to get the angle in radians (range: [-pi, pi]).
    # arctan2 is crucial as it correctly handles all four quadrants.
    angle_rad = np.arctan2(df[sin_col], df[cos_col])

    # 2. Scale back to the original period (e.g., [0, 360]).
    # Formula: (angle_rad / (2 * pi)) * period
    angle_normalized = angle_rad * (period / (2 * np.pi))

    # 3. Handle the cyclical nature to ensure the result is non-negative (e.g., [0, 360)).
    # The decoded value might be negative (e.g., -90 degrees). We shift it to the
    # equivalent positive value (e.g., 270 degrees).
    df[reconstructed_col] = np.where(angle_normalized < 0,
                                     angle_normalized + period,
                                     angle_normalized)

    print(f"Reconstructed column: '{reconstructed_col}'")
    return df

In [6]:
# ['Right Ascension of the Ascending Node (degrees)','Argument of Perigee (degrees)', 'Mean Anomaly (degrees)']
CYCLE_PERIOD = 360.0
FEATURE_COL = 'Right Ascension of the Ascending Node (degrees)'
data_for_training = encode_cyclical_feature(data_for_training, FEATURE_COL, CYCLE_PERIOD)

FEATURE_COL = 'Argument of Perigee (degrees)'
data_for_training = encode_cyclical_feature(data_for_training, FEATURE_COL, CYCLE_PERIOD)

FEATURE_COL = 'Mean Anomaly (degrees)'
data_for_training = encode_cyclical_feature(data_for_training, FEATURE_COL, CYCLE_PERIOD)


data_for_training

--- Encoding 'Right Ascension of the Ascending Node (degrees)' with period 360.0 ---
Created columns: 'Right Ascension of the Ascending Node (degrees)_sin' and 'Right Ascension of the Ascending Node (degrees)_cos'
--- Encoding 'Argument of Perigee (degrees)' with period 360.0 ---
Created columns: 'Argument of Perigee (degrees)_sin' and 'Argument of Perigee (degrees)_cos'
--- Encoding 'Mean Anomaly (degrees)' with period 360.0 ---
Created columns: 'Mean Anomaly (degrees)_sin' and 'Mean Anomaly (degrees)_cos'


,First Derivative Mean Motion,Inclination (degrees),Right Ascension of the Ascending Node (degrees),Argument of Perigee (degrees),Mean Anomaly (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch,Revolution Number at Epoch_Diff,Right Ascension of the Ascending Node (degrees)_sin,Right Ascension of the Ascending Node (degrees)_cos,Argument of Perigee (degrees)_sin,Argument of Perigee (degrees)_cos,Mean Anomaly (degrees)_sin,Mean Anomaly (degrees)_cos
Epoch,,,,,,,,,,,,,,,
2022-12-31,0.000157,51.644800,77.042350,208.400550,320.463100,0.000525,15.497978,37584.500000,NaN,0.974536,0.224231,-0.475633,-0.879644,-0.636575,0.771215
2023-01-01,0.000166,51.644567,74.151133,211.103567,277.293067,0.000519,15.498221,37593.666667,9.166667,0.961985,0.273101,-0.516587,-0.856235,-0.991910,0.126945
2023-01-02,0.000175,51.644680,69.554840,215.693640,221.644740,0.000511,15.498584,37608.000000,14.333333,0.937007,0.349311,-0.583451,-0.812148,-0.664510,-0.747279
2023-01-03,0.000162,51.644860,64.713420,219.864140,202.583880,0.000499,15.498874,37623.200000,15.200000,0.904183,0.427146,-0.640969,-0.767566,-0.384036,-0.923318
2023-01-04,0.000142,51.644740,59.072260,225.670000,219.529780,0.000497,15.499123,37640.800000,17.600000,0.857816,0.513957,-0.715327,-0.698790,-0.636479,-0.771294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,0.000261,51.638300,77.293500,11.883050,348.229450,0.000581,15.503139,48854.000000,15.200000,0.975510,0.219957,0.205915,0.978570,-0.203993,0.978972
2024-12-28,0.000264,51.638950,72.714000,15.958250,220.293583,0.000581,15.503644,48868.166667,14.166667,0.954833,0.297142,0.274937,0.961462,-0.646704,-0.762741
2024-12-29,0.000258,51.638957,67.260986,21.063200,221.625786,0.000583,15.504220,48885.142857,16.976190,0.922275,0.386534,0.359398,0.933185,-0.664263,-0.747499


In [7]:
data_for_training.dropna(inplace=True)

In [8]:
data_for_training.columns

Index(['First Derivative Mean Motion', 'Inclination (degrees)',
       'Right Ascension of the Ascending Node (degrees)',
       'Argument of Perigee (degrees)', 'Mean Anomaly (degrees)',
       'Eccentricity', 'Mean Motion (revolutions per day)',
       'Revolution Number at Epoch', 'Revolution Number at Epoch_Diff',
       'Right Ascension of the Ascending Node (degrees)_sin',
       'Right Ascension of the Ascending Node (degrees)_cos',
       'Argument of Perigee (degrees)_sin',
       'Argument of Perigee (degrees)_cos', 'Mean Anomaly (degrees)_sin',
       'Mean Anomaly (degrees)_cos'],
      dtype='object')

## Columns for training

In [9]:
gt_features = ['First Derivative Mean Motion', 'Inclination (degrees)',
       'Right Ascension of the Ascending Node (degrees)',
       'Argument of Perigee (degrees)', 'Mean Anomaly (degrees)',
       'Eccentricity', 'Mean Motion (revolutions per day)',
       'Revolution Number at Epoch']

In [10]:
raw_features = ['First Derivative Mean Motion', 'Inclination (degrees)','Eccentricity','Mean Motion (revolutions per day)']

diff_features = ['Revolution Number at Epoch_Diff']

cyclical_features = ['Right Ascension of the Ascending Node (degrees)_sin',
        'Right Ascension of the Ascending Node (degrees)_cos',
        'Argument of Perigee (degrees)_sin',
        'Argument of Perigee (degrees)_cos', 
        'Mean Anomaly (degrees)_sin',
        'Mean Anomaly (degrees)_cos']

In [11]:
all_features = raw_features+diff_features+cyclical_features

# Training models

In [12]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [13]:
data = data_for_training[all_features].copy()
data.head()

,First Derivative Mean Motion,Inclination (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch_Diff,Right Ascension of the Ascending Node (degrees)_sin,Right Ascension of the Ascending Node (degrees)_cos,Argument of Perigee (degrees)_sin,Argument of Perigee (degrees)_cos,Mean Anomaly (degrees)_sin,Mean Anomaly (degrees)_cos
Epoch,,,,,,,,,,,
2023-01-01,0.000166,51.644567,0.000519,15.498221,9.166667,0.961985,0.273101,-0.516587,-0.856235,-0.991910,0.126945
2023-01-02,0.000175,51.644680,0.000511,15.498584,14.333333,0.937007,0.349311,-0.583451,-0.812148,-0.664510,-0.747279
2023-01-03,0.000162,51.644860,0.000499,15.498874,15.200000,0.904183,0.427146,-0.640969,-0.767566,-0.384036,-0.923318
2023-01-04,0.000142,51.644740,0.000497,15.499123,17.600000,0.857816,0.513957,-0.715327,-0.698790,-0.636479,-0.771294
2023-01-05,0.000139,51.644820,0.000490,15.499368,13.800000,0.816115,0.577890,-0.754367,-0.656453,-0.944209,-0.329347


# Train LSTM

In [28]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [45]:
class TimeSeriesDataset(Dataset):
    """Dataset for time series data with sliding window."""
    
    def __init__(self, data, seq_length, pred_length):
        self.data = torch.FloatTensor(data)
        self.seq_length = seq_length
        self.pred_length = pred_length
        
    def __len__(self):
        return len(self.data) - self.seq_length - self.pred_length + 1
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_length]
        y = self.data[idx + self.seq_length:idx + self.seq_length + self.pred_length]
        return x, y


class LSTMModel(nn.Module):
    """2-layer LSTM model for time series forecasting."""
    
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # x shape: (batch, seq_len, input_size)
        lstm_out, _ = self.lstm(x)
        # Take the last output
        last_output = lstm_out[:, -1, :]
        prediction = self.fc(last_output)
        return prediction


class TimeSeriesLSTM:
    """Wrapper class for training LSTM on multiple time series."""
    
    def __init__(self, seq_length=30, pred_length=1, hidden_size=64, 
                 num_layers=2, dropout=0.2, learning_rate=0.001, 
                 batch_size=32, epochs=100, device=None, save_dir='LSTM_models'):
        """
        Args:
            seq_length: Length of input sequence (lookback window)
            pred_length: Length of prediction horizon
            hidden_size: Number of hidden units in LSTM
            num_layers: Number of LSTM layers (fixed at 2 for this implementation)
            dropout: Dropout rate
            learning_rate: Learning rate for optimizer
            batch_size: Batch size for training
            epochs: Number of training epochs
            device: Device to train on ('cuda' or 'cpu')
            save_dir: Directory to save trained models
        """
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.device = device if device else ('cuda' if torch.cuda.is_available() else 'cpu')
        self.save_dir = save_dir
        
        self.models = {}
        self.scalers = {}
        
        # Create save directory if it doesn't exist
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)
        
    def prepare_data(self, series):
        """Normalize data and create dataset."""
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(series.reshape(-1, 1)).flatten()
        dataset = TimeSeriesDataset(scaled_data, self.seq_length, self.pred_length)
        return dataset, scaler
    
    def train_single_series(self, series, column_name, verbose=True, show_progress=False):
        """Train LSTM model for a single time series."""
        # Prepare data
        dataset, scaler = self.prepare_data(series)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        # Initialize model
        model = LSTMModel(
            input_size=1,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            output_size=self.pred_length,
            dropout=self.dropout
        ).to(self.device)
        
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate)
        
        # Training loop with optional progress bar
        model.train()
        epoch_iterator = tqdm.tqdm(range(self.epochs), desc=f"Training {column_name}", 
                             disable=not show_progress, leave=False)
        
        for epoch in epoch_iterator:
            total_loss = 0
            for x_batch, y_batch in dataloader:
                x_batch = x_batch.unsqueeze(-1).to(self.device)
                y_batch = y_batch.to(self.device)
                
                optimizer.zero_grad()
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            avg_loss = total_loss / len(dataloader)
            
            if show_progress:
                epoch_iterator.set_postfix({'loss': f'{avg_loss:.6f}'})
            elif verbose and (epoch + 1) % 20 == 0:
                print(f"  Column '{column_name}' - Epoch [{epoch+1}/{self.epochs}], Loss: {avg_loss:.6f}")
        
        # Store model and scaler in memory
        self.models[column_name] = model
        self.scalers[column_name] = scaler
        
        # Save model to disk
        self.save_model(column_name)
        
    def save_model(self, column_name):
        """Save model and scaler for a specific column."""
        model_path = os.path.join(self.save_dir, f'{column_name}_model.pth')
        scaler_path = os.path.join(self.save_dir, f'{column_name}_scaler.pkl')
        
        # Save model state
        torch.save({
            'model_state_dict': self.models[column_name].state_dict(),
            'hyperparameters': {
                'input_size': 1,
                'hidden_size': self.hidden_size,
                'num_layers': self.num_layers,
                'output_size': self.pred_length,
                'dropout': self.dropout
            }
        }, model_path)
        
        # Save scaler
        import pickle
        with open(scaler_path, 'wb') as f:
            pickle.dump(self.scalers[column_name], f)
    
    def load_model(self, column_name):
        """Load model and scaler for a specific column."""
        model_path = os.path.join(self.save_dir, f'{column_name}_model.pth')
        scaler_path = os.path.join(self.save_dir, f'{column_name}_scaler.pkl')
        
        # Load model
        checkpoint = torch.load(model_path, map_location=self.device)
        model = LSTMModel(**checkpoint['hyperparameters']).to(self.device)
        model.load_state_dict(checkpoint['model_state_dict'])
        self.models[column_name] = model
        
        # Load scaler
        import pickle
        with open(scaler_path, 'rb') as f:
            self.scalers[column_name] = pickle.load(f)
        
    def fit(self, df, verbose=True, show_progress=True):
        """
        Train LSTM models for each column in the dataframe.
        
        Args:
            df: pandas DataFrame with time series data
            verbose: Print training progress
            show_progress: Show progress bars for training
        """
        if verbose:
            print(f"Training LSTM models on {len(df.columns)} time series...")
            print(f"Device: {self.device}")
            print(f"Models will be saved to: {self.save_dir}\n")
        
        # Progress bar for columns
#         column_iterator = tqdm.tqdm(df.columns, desc="Overall Progress", 
#                               disable=not show_progress)
        
        for col in df.columns:
#             if show_progress:
#                 column_iterator.set_description(f"Training column: {col}")
#             elif verbose:
            print(f"\nTraining model for column: {col}")
            
            series = df[col].values
            self.train_single_series(series, col, verbose=verbose, 
                                    show_progress=show_progress)
            
#             if verbose and show_progress:
#                 print(f"✓ Model for '{col}' trained and saved")
        
        if verbose:
            print(f"\n{'='*60}")
            print("Training complete! All models saved.")
            print(f"{'='*60}")
    
    def predict_single_series(self, series, column_name, n_steps=1):
        """Predict future values for a single time series."""
        model = self.models[column_name]
        scaler = self.scalers[column_name]
        model.eval()
        
        # Normalize input
        scaled_data = scaler.transform(series.reshape(-1, 1)).flatten()
        
        predictions = []
        current_seq = scaled_data[-self.seq_length:].copy()
        
        with torch.no_grad():
            for _ in range(n_steps):
                # Prepare input
                x = torch.FloatTensor(current_seq).unsqueeze(0).unsqueeze(-1).to(self.device)
                
                # Predict
                pred = model(x).cpu().numpy().flatten()
                predictions.extend(pred)
                
                # Update sequence for next prediction
                current_seq = np.roll(current_seq, -self.pred_length)
                current_seq[-self.pred_length:] = pred[:self.pred_length]
        
        # Inverse transform predictions
        predictions = np.array(predictions).reshape(-1, 1)
        predictions = scaler.inverse_transform(predictions).flatten()
        
        return predictions[:n_steps]
    
    def predict(self, df, n_steps=1, load_from_disk=False):
        """
        Predict future values for all columns in the dataframe.
        
        Args:
            df: pandas DataFrame with time series data (must match training columns)
            n_steps: Number of future steps to predict
            load_from_disk: If True, load models from disk before predicting
            
        Returns:
            pandas DataFrame with predictions for each column
        """
        predictions = {}
        
        for col in df.columns:
            # Load model from disk if requested
            if load_from_disk and col not in self.models:
                print(f"Loading model for '{col}' from disk...")
                self.load_model(col)
            
            series = df[col].values
            pred = self.predict_single_series(series, col, n_steps)
            predictions[col] = pred
        
        # Create output dataframe
        pred_df = pd.DataFrame(predictions)
        return pred_df

In [46]:
split_index = int(len(data) * 0.8)

train_df = data.iloc[:split_index,:].copy()
test_df = data.iloc[split_index:,:].copy()

train_df.shape, test_df.shape

((584, 11), (146, 11))

In [ ]:
data_for_training[gt_features]

In [57]:
test_df = data_for_training[gt_features].iloc[split_index:,:].copy()
test_df.shape

(146, 8)

In [67]:
# Initialize and train model
lstm = TimeSeriesLSTM(
    seq_length=3,
    pred_length=1,
    hidden_size=32,
    num_layers=1,
    dropout=0.2,
    learning_rate=0.001,
    batch_size=64,
    epochs=50
)

# Train on the entire dataframe
lstm.fit(train_df, verbose=True)



Training LSTM models on 11 time series...
Device: cpu
Models will be saved to: LSTM_models


Training model for column: First Derivative Mean Motion



Training model for column: Inclination (degrees)



Training model for column: Eccentricity



Training model for column: Mean Motion (revolutions per day)



Training model for column: Revolution Number at Epoch_Diff


...


Training model for column: Argument of Perigee (degrees)_sin



Training model for column: Argument of Perigee (degrees)_cos



Training model for column: Mean Anomaly (degrees)_sin



Training model for column: Mean Anomaly (degrees)_cos



Training complete! All models saved.


In [68]:
# Predict next 10 steps
predictions = lstm.predict(train_df, n_steps=146)

print("\n" + "="*50)
print("Predictions DataFrame shape:", predictions.shape)
print("\nPredicted values:")
print(predictions)


Predictions DataFrame shape: (146, 11)

Predicted values:
     First Derivative Mean Motion  Inclination (degrees)  Eccentricity  \
0                        0.000169              51.639469      0.000603   
1                        0.000166              51.639462      0.000604   
2                        0.000163              51.639492      0.000606   
3                        0.000159              51.639526      0.000612   
4                        0.000158              51.639549      0.000615   
..                            ...                    ...           ...   
141                      0.000154              51.640083      0.000930   
142                      0.000154              51.640083      0.000930   
143                      0.000154              51.640083      0.000930   
144                      0.000154              51.640083      0.000930   
145                      0.000154              51.640083      0.000930   

     Mean Motion (revolutions per day)  Revolution N

# Reconstruction

In [69]:
df_pred_decoded = decode_cyclical_feature(predictions.copy(),
                                         sin_col='Right Ascension of the Ascending Node (degrees)_sin',
                                         cos_col='Right Ascension of the Ascending Node (degrees)_cos',period=CYCLE_PERIOD)
df_pred_decoded = decode_cyclical_feature(df_pred_decoded,
                                         sin_col='Argument of Perigee (degrees)_sin',
                                         cos_col='Argument of Perigee (degrees)_cos',period=CYCLE_PERIOD)
df_pred_decoded = decode_cyclical_feature(df_pred_decoded,
                                         sin_col='Mean Anomaly (degrees)_sin',
                                         cos_col='Mean Anomaly (degrees)_cos',period=CYCLE_PERIOD)

reconstructed_cols = [ col for col in df_pred_decoded.columns if '_reconstructed' in col]
encoded_cols = [col for col in df_pred_decoded.columns if '_sin' in col or '_cos' in col]
 
# df_pred = df_pred_decoded[reconstructed_cols]
# df_pred.head()
# df_pred_decoded#[reconstructed_cols]
df_pred_decoded.drop(encoded_cols, axis=1, inplace=True)


--- Decoding from 'Right Ascension of the Ascending Node (degrees)_sin' and 'Right Ascension of the Ascending Node (degrees)_cos' ---
Reconstructed column: 'Right Ascension of the Ascending Node (degrees)_reconstructed'

--- Decoding from 'Argument of Perigee (degrees)_sin' and 'Argument of Perigee (degrees)_cos' ---
Reconstructed column: 'Argument of Perigee (degrees)_reconstructed'

--- Decoding from 'Mean Anomaly (degrees)_sin' and 'Mean Anomaly (degrees)_cos' ---
Reconstructed column: 'Mean Anomaly (degrees)_reconstructed'


In [70]:
initial_revs

46697.71428571428

In [71]:
initial_revs = data_for_training.iloc[split_index+2,7]#['Revolution Number at Epoch']
initial_revs

46697.71428571428

In [72]:
df_pred_decoded['Revolution Number at Epoch'] = df_pred_decoded['Revolution Number at Epoch_Diff'].cumsum() + initial_revs
df_pred_decoded.shape, df_pred_decoded.head()

((146, 9),
    First Derivative Mean Motion  Inclination (degrees)  Eccentricity  \
 0                      0.000169              51.639469      0.000603   
 1                      0.000166              51.639462      0.000604   
 2                      0.000163              51.639492      0.000606   
 3                      0.000159              51.639526      0.000612   
 4                      0.000158              51.639549      0.000615   
 
    Mean Motion (revolutions per day)  Revolution Number at Epoch_Diff  \
 0                          15.497594                        15.357750   
 1                          15.497815                        15.661796   
 2                          15.498004                        15.421877   
 3                          15.498178                        15.596713   
 4                          15.498334                        15.496282   
 
    Right Ascension of the Ascending Node (degrees)_reconstructed  \
 0                                

In [73]:
df_pred_data = df_pred_decoded[['First Derivative Mean Motion', 'Inclination (degrees)', 
        'Right Ascension of the Ascending Node (degrees)_reconstructed',
       'Argument of Perigee (degrees)_reconstructed',
       'Mean Anomaly (degrees)_reconstructed', 'Eccentricity', 'Mean Motion (revolutions per day)', 'Revolution Number at Epoch']].copy()
df_pred_data.head()

,First Derivative Mean Motion,Inclination (degrees),Right Ascension of the Ascending Node (degrees)_reconstructed,Argument of Perigee (degrees)_reconstructed,Mean Anomaly (degrees)_reconstructed,Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch
0,0.000169,51.639469,62.734604,166.035385,225.119064,0.000603,15.497594,46713.074219
1,0.000166,51.639462,60.508781,165.214371,220.343887,0.000604,15.497815,46728.734375
2,0.000163,51.639492,59.213028,163.436478,220.088867,0.000606,15.498004,46744.156250
3,0.000159,51.639526,58.577690,161.035248,216.272324,0.000612,15.498178,46759.753906
4,0.000158,51.639549,57.531143,159.008118,212.400833,0.000615,15.498334,46775.250000


In [74]:
y_actual = test_df.values
y_pred = df_pred_data.values

if y_actual.shape != y_pred.shape:
    raise ValueError(f"Actual and prediction arrays must have the same shape. Got {y_actual.shape} and {y_pred.shape}")

# Ensure arrays are at least 2D (even for a single feature)
if y_actual.ndim == 1:
    y_actual = y_actual.reshape(-1, 1)
    y_pred = y_pred.reshape(-1, 1)

num_features = y_actual.shape[1]

if gt_features is None:
    gt_features = [f'Feature_{i+1}' for i in range(num_features)]
elif len(gt_features) != num_features:
    raise ValueError(f"The number of feature names ({len(gt_features)}) must match the number of columns ({num_features}).")


metrics_data = {}

for i in range(num_features):
    actual_flat = y_actual[:, i].flatten()
    pred_flat = y_pred[:, i].flatten()

    # Calculate Error
    error = pred_flat - actual_flat

    # --- 1. MSE (Mean Squared Error) ---
    mse = np.mean(error ** 2)

    # --- 2. RMSE (Root Mean Squared Error) ---
    rmse = np.sqrt(mse)

    # --- 3. MAE (Mean Absolute Error) ---
    mae = np.mean(np.abs(error))

    # --- 4. MAPE (Mean Absolute Percentage Error) ---

    #     # Use a mask to avoid division by zero (or near-zero) for the denominator
    #     # We only consider actual values that are meaningfully non-zero
    #     valid_mask = np.abs(actual_flat) > epsilon

    #     if np.any(valid_mask):
    #         # Calculate the percentage error only for valid points
    #         percentage_error = np.abs(error[valid_mask] / actual_flat[valid_mask])
    #         mape = np.mean(percentage_error) * 100
    #     else:
    #         # If all actual values are near zero, MAPE is undefined/unreliable
    #         mape = np.nan 

    metrics_data[gt_features[i]] = {
        'RMSE': rmse,
        'MSE': mse,
        'MAE': mae,
#         'MAPE (%)': mape
    }

# Convert the dictionary of metrics into a DataFrame
results_df = pd.DataFrame.from_dict(metrics_data, orient='index')
results_df.index.name = 'Feature'

In [75]:
results_df

,RMSE,MSE,MAE
Feature,,,
First Derivative Mean Motion,0.000939,8.810142e-07,0.000245
Inclination (degrees),0.001733,3.004813e-06,0.001391
Right Ascension of the Ascending Node (degrees),163.433030,2.671036e+04,134.302698
Argument of Perigee (degrees),150.544529,2.266366e+04,120.281651
Mean Anomaly (degrees),71.492441,5.111169e+03,56.702632
Eccentricity,0.000183,3.340783e-08,0.000140
Mean Motion (revolutions per day),0.006134,3.762843e-05,0.004822
Revolution Number at Epoch,48.415395,2.344051e+03,48.390502
